In [ ]:
%env OPENAI_API_KEY="keyhere"

In [8]:
import logging

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

from kagent.tools.argo import (
    # Rollout Management
    ListRollouts,
    GetRollout,
    PauseRollout,
    PromoteRollout,
    SetRolloutImage,
    StatusRollout,
    # Installation & Verification
    VerifyArgoRolloutsControllerInstall,
    VerifyKubectlPluginInstall,
    CheckPluginLogsTool,
    VerifyGatewayPluginTool,
    # Resource Generation
    GenerateResource,
)

from kagent.tools.k8s import (
    ApplyManifest,
    GetResources,
    CreateResource,
    PatchResource,
    DeleteResource,
)

from kagent.tools.docs import Config, QueryTool

# Model configuration
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
)

# Documentation agent configuration
docs_config = Config(
    docs_path="",  # empty string means the database will be downloaded
)

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks that can be executed by the team members. DO NOT MAKE UP ADDITIONAL AND UNNECESSARY SUBTASKS.
    Your team members are:
        argo_agent: Performs Argo Rollouts tasks. This is only responsible for defining the YAML for the Argo Rollout resource and simple kubectl argo rollouts commands. It does not apply the config.
        k8s_agent: Performs Kubernetes tasks, including applying manifests to the cluster.

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

# Define tool groups for better organization
k8s_tools = [CreateResource(), GetResources(), ApplyManifest(), PatchResource(), DeleteResource()]

argo_management_tools = [
    ListRollouts(),
    GetRollout(),
    PauseRollout(),
    PromoteRollout(),
    SetRolloutImage(),
    StatusRollout(),
]

argo_setup_tools = [
    VerifyKubectlPluginInstall(),
    VerifyArgoRolloutsControllerInstall(),
    CheckPluginLogsTool(),
    VerifyGatewayPluginTool(),
    GenerateResource(),
]

K8S_SYSTEM_MESSAGE = """You are a Kubernetes specialist agent responsible for cluster operations and resource management.

Key Responsibilities:
1. Resource Management
   - Monitor and manage Kubernetes resources (pods, services, deployments)
   - Apply and validate Kubernetes manifests
   - Ensure proper resource state and health

2. Cluster Operations
   - Verify cluster connectivity and health
   - Monitor resource utilization
   - Report on cluster state

Guidelines:
- Always verify resource existence before operations
- Use proper error handling for kubectl operations
- Provide clear feedback on operation status
- Always attempt to apply the manifest to the cluster
- If no explicit tasks remain, return TERMINATE

Example interactions:
- "Get pods in namespace": List all pods with their status
- "Apply manifest": Validate and apply provided manifest
- "Check service": Verify service existence and endpoints
"""

ARGO_SYSTEM_MESSAGE = """You are an Argo Rollouts specialist focused on progressive delivery and deployment automation. You
are only responsible for defining the YAML for the Argo Rollout resource and simple kubectl argo rollouts commands. You do not apply
resources to the cluster.

Key Responsibilities:
1. Installation & Setup
   - Verify kubectl plugin and controller installation
   - Guide through initial setup process
   - Validate system requirements

2. Rollout Management
   - Create and configure rollouts
   - Manage canary deployments
   - Handle promotion and rollback
   - Monitor rollout status

3. Integration
   - Configure traffic management (Istio/Gateway API)
   - Set up metrics-based analysis
   - Integrate with existing deployment workflows

Best Practices:
- Always verify prerequisites before operations
- Provide step-by-step guidance for complex tasks
- Ensure proper validation at each stage
- Document any configuration changes

Example commands:
- `kubectl argo rollouts get rollout <name>`
- `kubectl argo rollouts promote <name>`
- `kubectl argo rollouts status <name>`
"""

DOCS_SYSTEM_MESSAGE = """You are a CNCF documentation specialist providing accurate information from official sources.

Key Responsibilities:
1. Documentation Search
   - Search across supported products: 'kubernetes', 'istio', 'argo cd', 'argo rollouts', 'helm', 'prometheus'
   - Provide accurate, relevant information
   - Include documentation links

2. Information Analysis
   - Understand context of queries
   - Cross-reference multiple sources
   - Provide comprehensive answers

Guidelines:
- Always verify product context before searching
- Provide direct documentation links
- Explain complex concepts clearly
- Cross-reference related documentation

Process for Each Query:
1. Identify target product
2. Search relevant documentation
3. Analyze and synthesize information
4. Provide clear, documented response
"""

# Create agents with improved configurations
k8s_agent = AssistantAgent(
    "k8s_agent",
    description="Kubernetes specialist for cluster operations and resource management",
    tools=k8s_tools,
    model_client=model_client,
    system_message=K8S_SYSTEM_MESSAGE,
)

argo_agent = AssistantAgent(
    "argo_agent",
    description="Argo Rollouts specialist for progressive delivery management.",
    tools=argo_management_tools + argo_setup_tools,
    model_client=model_client,
    system_message=ARGO_SYSTEM_MESSAGE,
)

docs_agent = AssistantAgent(
    "cncf_docs_agent",
    description="CNCF documentation specialist for accurate information retrieval",
    tools=[QueryTool(config=docs_config)],
    model_client=model_client,
    system_message=DOCS_SYSTEM_MESSAGE,
)

# Configure team with improved termination conditions
team = RoundRobinGroupChat(
    max_turns=5,  # Increased from 3 to allow for more complex interactions
    participants=[planning_agent, k8s_agent, argo_agent],
    termination_condition=TextMentionTermination(text="TERMINATE"),
)


# Example usage
async def run_team_task(task: str):
    """Run a task through the multi-agent team with proper error handling. If you don't have any explicit tasks left to complete, return TERMINATE."""
    try:
        return await Console(team.run_stream(task=task))
    except Exception as e:
        logging.error(f"Error executing task: {e}")
        return None


# Usage example:
# task = "Help me switch to Argo Rollouts in my cluster?"
# task = "Help me use Kubernetes Gateway APIs with argo rollouts in my cluster?"
task = "Translate my current reviews-v1 Kubernetes deployments in the default namespace to an Argo Rollout using the canary strategy and apply it to the cluster."
await run_team_task(task)

---------- user ----------
Translate my current reviews-v1 Kubernetes deployments in the default namespace to an Argo Rollout using the canary strategy and apply it to the cluster.
---------- PlanningAgent ----------
To translate your current Kubernetes deployment to an Argo Rollout using the canary strategy and apply it, we'll follow these steps:

1. Retrieve the existing deployment manifest.
2. Convert the deployment manifest to an Argo Rollout manifest.
3. Apply the Argo Rollout to the cluster.

Here's the breakdown of tasks:

1. k8s_agent: Retrieve the current `reviews-v1` deployment manifest from the default namespace.
2. argo_agent: Transform the retrieved deployment manifest into an Argo Rollout manifest with a canary strategy.
3. k8s_agent: Apply the Argo Rollout manifest to the cluster.

Please proceed with these tasks.
---------- k8s_agent ----------
[FunctionCall(id='call_dCyoR3jvLgFMC58lRoy7FQIc', arguments='{"name":"reviews-v1","resource_type":"deployment","all_namespaces"

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Translate my current reviews-v1 Kubernetes deployments in the default namespace to an Argo Rollout using the canary strategy and apply it to the cluster.', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=211, completion_tokens=137), content="To translate your current Kubernetes deployment to an Argo Rollout using the canary strategy and apply it, we'll follow these steps:\n\n1. Retrieve the existing deployment manifest.\n2. Convert the deployment manifest to an Argo Rollout manifest.\n3. Apply the Argo Rollout to the cluster.\n\nHere's the breakdown of tasks:\n\n1. k8s_agent: Retrieve the current `reviews-v1` deployment manifest from the default namespace.\n2. argo_agent: Transform the retrieved deployment manifest into an Argo Rollout manifest with a canary strategy.\n3. k8s_agent: Apply the Argo Rollout manifest to the cluster.\n\nPlease proceed with these task

In [ ]:
print(team.dump_component().model_dump_json(indent=2))